### installs

In [1]:
# installs
# python3 -m pip3 install numpy
# python3 -m pip3 install pandas
# python3 -m pip3 install shapely
# brew install gdal # fiona dependency
# python3 -m pip3 install fiona # geopandas dependency
# python3 -m pip3 install pyproj # geopandas dependency
# python3 -m pip3 install pygeos # geopandas dependency
# python3 -m pip3 install geopandas
# python3 -m pip install jupyter
# python3 -m pip3 install folium
# python3 -m pip3 install matplotlib
# python3 -m pip3 install seaborn
# python3 -m pip install tqdm

### imports

In [2]:
# imports
import pandas as pd
import numpy as np
import glob
import os
# import datetime as dt
# import pytz # python timezones
from pathlib import Path
import re
import time
from tqdm.auto import tqdm
import pickle
import shapely.wkt
from shapely.geometry import Point, Polygon
import geopandas as gpd
from geopandas.tools import sjoin

/Users/robertbozsik/techtest/adsquare/adsquare_env/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### user-defined functions

In [4]:
def date_from_utc_ms_ts(utc_ms_ts) -> str:
    """Return a date (yyyy-mm-dd) from a string of utc timestamp in milliseconds (timezone = Europe/Berlin).

    :param utc_ms_ts: Unix UTC timestamp in milliseconds (int or str)
    :return: date yyyy-mm-dd (str)
    """
    # convert from time stamp to datetime
    utc_datetime = dt.datetime.utcfromtimestamp(int(utc_ms_ts) / 1000)
    # set the timezone to UTC, and then convert to desired timezone
    date = (utc_datetime
            .replace(tzinfo=pytz.timezone('UTC'))
            .astimezone(pytz.timezone('Europe/Berlin'))
            .strftime('%Y-%m-%d'))
    return date

# 1. Analyze the store visitation by date and affinity profile of store visitors.

## 1.a Resolve the user visits per store, i.e. filter the GPS signals through polygons.

### stores

In [5]:
stores = pd.read_csv("../../assignment_data/stores.csv")

In [6]:
# stores.shape

In [7]:
stores.head()

,store_id,store_name,wkt
0,place_1,McDonald's,POLYGON ((13.4611920000000005 52.4709870000000...
1,place_2,McDonald's,POLYGON ((13.4683480000000007 52.5471599999999...
2,place_3,McDonald's,POLYGON ((13.3128810000000009 52.4197929999999...
3,place_4,McDonald's,POLYGON ((13.3635780000000004 52.5606039999999...
4,place_5,McDonald's,POLYGON ((13.3230909999999998 52.5613460000000...


In [8]:
# transform strings to polygons in column "wkt"
stores["wkt"] = stores["wkt"].apply(lambda x: shapely.wkt.loads(x))

In [9]:
# rename "wkt" to "geometry"
stores = stores.rename(columns={"wkt": "geometry"}) # must be geometry for the geopandas join

In [10]:
# type(stores["geometry"][0])

In [11]:
# transform stores into a GeoDataFrame
stores_gdf = gpd.GeoDataFrame(stores)

In [12]:
stores_gdf.head()

,store_id,store_name,geometry
0,place_1,McDonald's,"POLYGON ((13.46119 52.47099, 13.46103 52.47066..."
1,place_2,McDonald's,"POLYGON ((13.46835 52.54716, 13.46841 52.54716..."
2,place_3,McDonald's,"POLYGON ((13.31288 52.41979, 13.31303 52.41966..."
3,place_4,McDonald's,"POLYGON ((13.36358 52.56060, 13.36358 52.56061..."
4,place_5,McDonald's,"POLYGON ((13.32309 52.56135, 13.32307 52.56128..."


In [13]:
stores_gdf.shape

(247, 3)

In [14]:
type(stores_gdf)

geopandas.geodataframe.GeoDataFrame

In [15]:
# save stores_gdf as geojson for visualization in Tableau
# stores_gdf.to_file("../out_data_full/stores_gdf.geojson", driver="GeoJSON")

In [16]:
# save stores_gdf as pickle
# stores_gdf.to_pickle("../out_data_full/stores_gdf.pkl")

### gps signals

**read and transform gps signal batches**

In [17]:
signals_gdf_list = []

In [18]:
# start = time.time()
# print("Reading and transforming gps signal csv batches...")

# path_sample = "../../assignment_data/full_data/"

# for index, file_name in enumerate(glob.glob(path_sample + "*.csv")):
#     # open a partition/batch of a gps signal as a Pandas DataFrame
#     signal = pd.read_csv(path_sample + f"part_{index + 1}.csv")
#     # sort by "utc_timestamp" ascending
#     signal = signal.sort_values(by=["utc_timestamp"]).reset_index(drop=True)
#     # transform utc_timestamp into yyyy-mm-dd (Europe/Berlin timezone)
#     # signal["utc_timestamp"] = signal["utc_timestamp"].apply(lambda x: date_from_utc_ms_ts(x))
#     signal["utc_timestamp"] = signal["utc_timestamp"].astype("datetime64[ms]").dt.to_period("D")
#     # rename "utc_timestamp" to "date"
#     signal = signal.rename(columns={"utc_timestamp": "date"})
#     # create POINT from lat lon and transform df into gdf
#     signal_gdf = gpd.GeoDataFrame(signal, geometry=gpd.points_from_xy(signal["lon"], signal["lat"]))
#     # append into the signals_list
#     signals_gdf_list.append(signal_gdf)

# end = time.time()
# dt = end - start
# print(f"The above task took {round(dt/60, 2)} minute(s).")

In [19]:
start = time.time()
print("Reading and transforming gps signal csv batches into GeoDataFrames...")

path_sample = "../../assignment_data/full_data/"
files_list = glob.glob(path_sample + "*.csv")

for index in tqdm(range(len(files_list))):
    # open a partition/batch of a gps signal as a Pandas DataFrame
    signal = pd.read_csv(path_sample + f"part_{index + 1}.csv")
    # sort by "utc_timestamp" ascending
    signal = signal.sort_values(by=["utc_timestamp"]).reset_index(drop=True)
    # transform utc_timestamp into yyyy-mm-dd (Europe/Berlin timezone)
    # signal["utc_timestamp"] = signal["utc_timestamp"].apply(lambda x: date_from_utc_ms_ts(x))
    signal["utc_timestamp"] = signal["utc_timestamp"].astype("datetime64[ms]").dt.to_period("D")
    # rename "utc_timestamp" to "date"
    signal = signal.rename(columns={"utc_timestamp": "date"})
    # create POINT from lat lon and transform df into gdf
    signal_gdf = gpd.GeoDataFrame(signal, geometry=gpd.points_from_xy(signal["lon"], signal["lat"]))
    # append into the signals_list
    signals_gdf_list.append(signal_gdf)

end = time.time()
dt = end - start
print(f"The above task took {round(dt/60, 2)} minute(s).")

Reading and transforming gps signal csv batches...


  0%|          | 0/57 [00:00<?, ?it/s]

The above task took 1.63 minute(s).


In [20]:
len(signals_gdf_list)

57

In [21]:
signals_gdf_list[0].head()

,device_id,lat,lon,date,geometry
0,14372,52.486812,13.395102,2021-01-01,POINT (13.39510 52.48681)
1,8853,52.542976,13.602809,2021-01-01,POINT (13.60281 52.54298)
2,22268,52.568431,13.523719,2021-01-01,POINT (13.52372 52.56843)
3,5914,52.567700,13.451360,2021-01-01,POINT (13.45136 52.56770)
4,5752,52.539409,13.519975,2021-01-01,POINT (13.51998 52.53941)


In [22]:
signals_gdf_list[0].shape

(1000000, 5)

In [23]:
signals_gdf_list[56].head()

,device_id,lat,lon,date,geometry
0,49649,52.669650,13.549480,2021-01-21,POINT (13.54948 52.66965)
1,43538,52.422310,13.474727,2021-01-21,POINT (13.47473 52.42231)
2,142367,52.520010,13.404950,2021-01-21,POINT (13.40495 52.52001)
3,41903,52.435479,13.529791,2021-01-21,POINT (13.52979 52.43548)
4,45678,52.522173,13.455405,2021-01-21,POINT (13.45540 52.52217)


In [24]:
signals_gdf_list[56].shape

(572824, 5)

### create spatial joins between stores_gdf and each signal_gdf and concat all spatial joins vertically together

In [25]:
signals_stores_gdf = gpd.GeoDataFrame()

In [26]:
# start = time.time()
# print("Joining each signal_gdf and stores_gdf...")

# for signal_gdf in signals_gdf_list:
#     # spatial join signal_gdf and stores_gdf
#     signals_stores = sjoin(signal_gdf, stores_gdf, how="inner")
#     # concat/add all spatial joins vertically together
#     signals_stores_gdf = pd.concat([signals_stores_gdf, signals_stores], ignore_index=True)

# end = time.time()
# dt = end - start
# print(f"The above task took {round(dt/60, 2)} minute(s).")

In [27]:
start = time.time()
print("Joining each signal_gdf and stores_gdf and creating a GeoDataFrame...")

for index in tqdm(range(len(signals_gdf_list))):
    signal_gdf = signals_gdf_list[index]
    # spatial join signal_gdf and stores_gdf
    signals_stores = sjoin(signal_gdf, stores_gdf, how="inner")
    # concat/add all spatial joins vertically together
    signals_stores_gdf = pd.concat([signals_stores_gdf, signals_stores], ignore_index=True)

end = time.time()
dt = end - start
print(f"The above task took {round(dt/60, 2)} minute(s).")

Joining each signal_gdf and stores_gdf...


  0%|          | 0/57 [00:00<?, ?it/s]

The above task took 1.81 minute(s).


In [28]:
signals_stores_gdf.shape

(68179, 8)

In [29]:
signals_stores_gdf.head()

,device_id,lat,lon,date,geometry,index_right,store_id,store_name
0,2022,52.505726,13.439094,2021-01-01,POINT (13.43909 52.50573),232,place_233,Mercedes
1,14375,52.520300,13.386380,2021-01-01,POINT (13.38638 52.52030),27,place_28,McDonald's
2,838,52.520299,13.386378,2021-01-01,POINT (13.38638 52.52030),27,place_28,McDonald's
3,20940,52.547120,13.468410,2021-01-01,POINT (13.46841 52.54712),1,place_2,McDonald's
4,20940,52.547120,13.468410,2021-01-01,POINT (13.46841 52.54712),1,place_2,McDonald's


In [30]:
signals_stores_gdf.tail()

,device_id,lat,lon,date,geometry,index_right,store_id,store_name
68174,142681,52.483676,13.377726,2021-01-21,POINT (13.37773 52.48368),155,place_156,Rewe
68175,142681,52.483624,13.377775,2021-01-21,POINT (13.37777 52.48362),155,place_156,Rewe
68176,142681,52.483604,13.377489,2021-01-21,POINT (13.37749 52.48360),155,place_156,Rewe
68177,142681,52.483606,13.377411,2021-01-21,POINT (13.37741 52.48361),155,place_156,Rewe
68178,142681,52.483770,13.377451,2021-01-21,POINT (13.37745 52.48377),155,place_156,Rewe


In [31]:
type(signals_stores_gdf)

geopandas.geodataframe.GeoDataFrame

### users and user affinities

**users**

For an inner join between signals_stores_gdf and user affinities I only need the affinity of those users who are in the signals_stores_gdf geopandas dataframe and not all the unique user affinities.

In [32]:
# create unique users by dropping each duplicated device_id from the signals_stores_gdf
users = signals_stores_gdf[["device_id"]].drop_duplicates(subset=["device_id"]).reset_index(drop=True)

In [33]:
users.tail()

,device_id
7559,41159
7560,46906
7561,129883
7562,47230
7563,45914


In [34]:
# sort by device_id ascending
users = users.sort_values(by=["device_id"]).reset_index(drop=True)

In [35]:
users.shape

(7564, 1)

In [36]:
users.head()

,device_id
0,2
1,5
2,8
3,16
4,29


In [37]:
users.tail()

,device_id
7559,172082
7560,172278
7561,172321
7562,172428
7563,173974


**user affinities**

In [38]:
# list the files in the user_affinities folder
path = os.getcwd()
# path = "/Users/robertbozsik/techtest/adsquare/assignment_data/affinities"
path = f"{str(Path(path).parents[1])}/assignment_data/affinities"
file_names = os.listdir(path)
file_names = sorted(file_names)
# file_names

In [39]:
# add .csv after the file names (it should be run only once)
if ".csv" not in file_names[0]:
    for index, file_name in enumerate(file_names):
        os.rename(os.path.join(path, file_name), os.path.join(path, "".join([file_name, ".csv"])))

In [40]:
# create a dictionary "user_affinities", key: value -> names-of-the-affinity: [lists-of-the-affinity-csv]
def list_from_affinities(aff_name: str) -> list:
    """Return a list created from the given user_affinity csv file"""
    aff_name = pd.read_csv(f"../../assignment_data/affinities/{aff_name}.csv", header=None, names=[aff_name])
    aff_name = list(aff_name.iloc[:, 0]) # all rows, first column
    return aff_name

user_affinities = {}

for file_name in file_names:
    name = file_name.split(".")[0]
    user_affinities[name] = list_from_affinities(name)

In [41]:
user_affinities.keys()

dict_keys(['addidas', 'apple', 'bmw', 'employed', 'female', 'h_&_m', 'high_income', 'honda', 'job_seeking', 'low_income', 'male', 'mercedes-benz', 'middle_income', 'retired', 'student', 'tommy_helfinger'])

In [42]:
# len(user_affinities["addidas"])

In [43]:
# len(user_affinities["low_income"])

In [44]:
# len(user_affinities["retired"])

**add user_affinities to users**

In [45]:
# try it out with the affinity "addidas"
users["addidas"] = np.where(users["device_id"].isin(user_affinities["addidas"]), 1, 0)

In [46]:
# users["addidas"].value_counts()

In [47]:
for index, key in enumerate(user_affinities.keys()):
    if index > 0: # the column "addidas" has already been created
        users[key] = np.where(users["device_id"].isin(user_affinities[key]), 1, 0)

In [48]:
users.shape

(7564, 17)

In [49]:
users.head()

,device_id,addidas,apple,bmw,employed,female,h_&_m,high_income,honda,job_seeking,low_income,male,mercedes-benz,middle_income,retired,student,tommy_helfinger
0,2,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0
1,5,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0
2,8,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0
3,16,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0
4,29,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0


In [50]:
# save users as csv
users.to_csv("../out_data_full/users.csv", index=False)

### merge signals_stores_gdf and users (affinities)

In [51]:
# merge gps_sig_and_stores and users (affinities)
signals_stores_gdf.shape

(68179, 8)

In [52]:
signals_stores_gdf.head()

,device_id,lat,lon,date,geometry,index_right,store_id,store_name
0,2022,52.505726,13.439094,2021-01-01,POINT (13.43909 52.50573),232,place_233,Mercedes
1,14375,52.520300,13.386380,2021-01-01,POINT (13.38638 52.52030),27,place_28,McDonald's
2,838,52.520299,13.386378,2021-01-01,POINT (13.38638 52.52030),27,place_28,McDonald's
3,20940,52.547120,13.468410,2021-01-01,POINT (13.46841 52.54712),1,place_2,McDonald's
4,20940,52.547120,13.468410,2021-01-01,POINT (13.46841 52.54712),1,place_2,McDonald's


In [53]:
users.shape

(7564, 17)

In [54]:
users.head()

,device_id,addidas,apple,bmw,employed,female,h_&_m,high_income,honda,job_seeking,low_income,male,mercedes-benz,middle_income,retired,student,tommy_helfinger
0,2,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0
1,5,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0
2,8,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0
3,16,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0
4,29,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0


In [55]:
# merge gps_sig_and_stores and users (affinities)
start = time.time()
print("Merging signals_stores_gdf and users (affinities)...")

signals_stores_useraff = signals_stores_gdf.merge(users, how="inner", on="device_id")

end = time.time()
dt = end - start
print(f"The above task took {round(dt/60, 2)} minute(s).")

Merging signals_stores_gdf and users (affinities)...
The above task took 0.0 minute(s).


In [56]:
signals_stores_useraff.shape

(68179, 24)

In [57]:
signals_stores_useraff.head()

,device_id,lat,lon,date,geometry,index_right,store_id,store_name,addidas,apple,bmw,employed,female,h_&_m,high_income,honda,job_seeking,low_income,male,mercedes-benz,middle_income,retired,student,tommy_helfinger
0,2022,52.505726,13.439094,2021-01-01,POINT (13.43909 52.50573),232,place_233,Mercedes,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1
1,14375,52.520300,13.386380,2021-01-01,POINT (13.38638 52.52030),27,place_28,McDonald's,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
2,14375,52.520050,13.388070,2021-01-16,POINT (13.38807 52.52005),28,place_29,McDonald's,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
3,14375,52.431890,13.547070,2021-01-18,POINT (13.54707 52.43189),24,place_25,McDonald's,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
4,838,52.520299,13.386378,2021-01-01,POINT (13.38638 52.52030),27,place_28,McDonald's,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0


### create users_in_stores for visualizing on a map

In [58]:
users_in_stores = signals_stores_useraff[["device_id", "lat", "lon", "date", "store_id", "store_name"]]

In [59]:
users_in_stores.head()

,device_id,lat,lon,date,store_id,store_name
0,2022,52.505726,13.439094,2021-01-01,place_233,Mercedes
1,14375,52.520300,13.386380,2021-01-01,place_28,McDonald's
2,14375,52.520050,13.388070,2021-01-16,place_29,McDonald's
3,14375,52.431890,13.547070,2021-01-18,place_25,McDonald's
4,838,52.520299,13.386378,2021-01-01,place_28,McDonald's


In [60]:
users_in_stores.to_csv("../out_data_full/users_in_stores.csv", index=False)

## 1.b Group the resolved visits by date (yyyy-mm-dd), store_name, and store_id.

## 1.c For each store_id/store_name/date provide the following metric.

### 1.c.i A total number of GPS signals per place_id/date.

### 1.c.ii A total number of unique visitors (i.e. device ids).

In [61]:
# create total number of GPS signals per place_id/date and total number of unique visitors
ssu_total_and_unique = (signals_stores_useraff
                        .groupby(by=["date", "store_name", "store_id"])
                        .agg({"lat": "count", "device_id": "nunique"})
                        .rename(columns={"lat": "total_signals", "device_id": "unique_visits"})
                        .reset_index())

In [62]:
ssu_total_and_unique.shape

(3107, 5)

In [63]:
ssu_total_and_unique.head()

,date,store_name,store_id,total_signals,unique_visits
0,2021-01-01,Aldi,place_135,2,1
1,2021-01-01,Aldi,place_64,5,3
2,2021-01-01,Aldi,place_72,5,1
3,2021-01-01,Aldi,place_79,10,2
4,2021-01-01,Aldi,place_84,1,1


In [64]:
ssu_total_and_unique.tail()

,date,store_name,store_id,total_signals,unique_visits
3102,2021-01-21,Rewe,place_211,17,4
3103,2021-01-21,Rewe,place_212,10,4
3104,2021-01-21,Sparkasse,place_246,4,1
3105,2021-01-21,Subway,place_52,6,1
3106,2021-01-21,Subway,place_53,3,2


### 1.c.iii A total number of unique visitors belonging to each affinity group.

In [65]:
users.columns

Index(['device_id', 'addidas', 'apple', 'bmw', 'employed', 'female', 'h_&_m',
       'high_income', 'honda', 'job_seeking', 'low_income', 'male',
       'mercedes-benz', 'middle_income', 'retired', 'student',
       'tommy_helfinger'],
      dtype='object')

In [66]:
# total number of unique visitors belonging to each affinity group
start = time.time()
print("Creating total number of unique visitors belonging to each affinity group...")

ssu_unique_aff = (signals_stores_useraff
                  .drop(["lat", "lon", "geometry", "index_right"], axis=1)
                  .drop_duplicates(subset=["date", "store_name", "store_id", "device_id"])
                  .groupby(by=["date", "store_name", "store_id"])
                  .agg(sum) # would sum lat, lon and device_id as well what is nonsense!!!
                  .reset_index()
                  .drop(["device_id"], axis=1))

end = time.time()
dt = end - start
print(f"The above task took {round(dt/60, 2)} minutes.")

Creating total number of unique visitors belonging to each affinity group...
The above task took 0.1 minutes.


In [67]:
ssu_unique_aff.shape

(3107, 19)

In [68]:
ssu_unique_aff.head()

,date,store_name,store_id,addidas,apple,bmw,employed,female,h_&_m,high_income,honda,job_seeking,low_income,male,mercedes-benz,middle_income,retired,student,tommy_helfinger
0,2021-01-01,Aldi,place_135,0,0,0,0,0,0,0,1,0,0,1,1,1,0,0,0
1,2021-01-01,Aldi,place_64,0,0,0,0,0,0,0,1,0,2,3,0,1,0,0,0
2,2021-01-01,Aldi,place_72,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0
3,2021-01-01,Aldi,place_79,0,0,0,0,0,1,0,0,0,0,2,0,2,0,0,0
4,2021-01-01,Aldi,place_84,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0


In [69]:
# delete unnecessary columns
ssu_unique_aff = ssu_unique_aff.drop(["date", "store_name", "store_id"], axis=1)

In [70]:
ssu_unique_aff.head()

,addidas,apple,bmw,employed,female,h_&_m,high_income,honda,job_seeking,low_income,male,mercedes-benz,middle_income,retired,student,tommy_helfinger
0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,0,0
1,0,0,0,0,0,0,0,1,0,2,3,0,1,0,0,0
2,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0
3,0,0,0,0,0,1,0,0,0,0,2,0,2,0,0,0
4,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0


### concat the ssu_total_and_unique and ssu_unique_aff

In [71]:
# concat the gsu_total_and_unique and gsu_unique_aff
final_df = pd.concat([ssu_total_and_unique, ssu_unique_aff], axis=1)

In [72]:
final_df.shape

(3107, 21)

In [73]:
final_df.head()

,date,store_name,store_id,total_signals,unique_visits,addidas,apple,bmw,employed,female,h_&_m,high_income,honda,job_seeking,low_income,male,mercedes-benz,middle_income,retired,student,tommy_helfinger
0,2021-01-01,Aldi,place_135,2,1,0,0,0,0,0,0,0,1,0,0,1,1,1,0,0,0
1,2021-01-01,Aldi,place_64,5,3,0,0,0,0,0,0,0,1,0,2,3,0,1,0,0,0
2,2021-01-01,Aldi,place_72,5,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0
3,2021-01-01,Aldi,place_79,10,2,0,0,0,0,0,1,0,0,0,0,2,0,2,0,0,0
4,2021-01-01,Aldi,place_84,1,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0


In [74]:
# save final df as csv
final_df.to_csv("../out_data_full/analysis.csv.", index=False)
print("final_df saved as csv")

final_df saved as csv


In [75]:
# Took about 3 minutes 20 seconds :)